# Linear Regression: Predicting Taxi-Out Delay
## Given the dataset, predict the runway time of the flight.

#### Task Details
1. Go through the dataset and perform preprocessing and then perform a 90:10 split for train and test purposes.
2. Firstly label encode the columns which are required.
3. Your target or **y variable is TAXI-OUT time**. Use all 8 algorithms above on the dataset with loss score as RMSE (Root mean Square Error).
4. Now, One-Hot encode all the data points and perform the 3rd Step again.

Keep in mind that you will be using the same splitted dataframe for all the training and testing and should not split again.

##### Some models to consider:-

Linear Models:-

1. Linear Regression
2. Ridge Regression(Popularily L1)
3. Lasso Regression(Popularily L2)

Non linear Models:-

1. KNN model
2. SVR
3. Naive Bayes
4. Random Forest
5. LightGBM(Tree Based Model)

##### Expected Submission
Submit a notebook to see if label encoding or one hot encoding is better for the model and which out of the 8 algorithms which is the best. Just a comparative study and plots to understand the results on bigger datasets.

##### Evaluation
Having a complete report along with comparative graphs.

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Configuring the Data Frame
> **Source:** MUHAMMET ALI BÜYÜKNACAR'S [work](https://www.kaggle.com/buyuknacar/jfk-flight-notebook). 

In [43]:
pd.set_option('display.width', 1200)
pd.set_option('display.max_columns', 25)

### Loading the data from csv file

In [44]:
taxi = pd.read_csv('M1_final.csv')
taxi.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,DEP_TIME_M,CRS_ARR_M,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,323,448,48,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,333,531,48,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,341,482,48,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,343,513,48,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,356,499,46,32,58,W,24,35,29.91,Fair / Windy,9,17,13


### Editing the column names for clarity
> **Source:** MUHAMMET ALI BÜYÜKNACAR'S [work](https://www.kaggle.com/buyuknacar/jfk-flight-notebook). 

In [45]:
col_names = {"OP_UNIQUE_CARRIER":"CARRIER_CODE",
                "TAIL_NUM":"FLIGHT_NO",
                "CRS_ELAPSED_TIME":"SCHEDULED_DURATION",
                "CRS_DEP_M":"SCHEDULED_DEPARTURE",
                "DEP_TIME_M":"ACTUAL_DEP_TIME",
                "CRS_ARR_M":"SCHEDULED_ARRIVAL",
                "sch_dep":"FLT_SCH_ARRIVAL",
                "sch_arr":"FLT_SCH_DEPARTURE"}

taxi = taxi.rename(col_names, axis=1)
taxi.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEST,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,323,448,48,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,333,531,48,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,341,482,48,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,343,513,48,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,356,499,46,32,58,W,24,35,29.91,Fair / Windy,9,17,13


### Preparing the Data for Analysis

### Checking for and dropping missing values

In [46]:
taxi.isnull().sum() # will show the total number of missing values for each feature

MONTH                  0
DAY_OF_MONTH           0
DAY_OF_WEEK            0
CARRIER_CODE           0
FLIGHT_NO              0
DEST                   0
DEP_DELAY              0
SCHEDULED_DURATION     0
DISTANCE               0
SCHEDULED_DEPARTURE    0
ACTUAL_DEP_TIME        0
SCHEDULED_ARRIVAL      0
Temperature            0
Dew Point              0
Humidity               0
Wind                   2
Wind Speed             0
Wind Gust              0
Pressure               0
Condition              0
FLT_SCH_ARRIVAL        0
FLT_SCH_DEPARTURE      0
TAXI_OUT               0
dtype: int64

In [47]:
taxi = taxi.dropna() # drop rows with missing values, reassign the dataframe with no null values as taxi
taxi.isnull().sum() # check to see the missing values have been dropped

MONTH                  0
DAY_OF_MONTH           0
DAY_OF_WEEK            0
CARRIER_CODE           0
FLIGHT_NO              0
DEST                   0
DEP_DELAY              0
SCHEDULED_DURATION     0
DISTANCE               0
SCHEDULED_DEPARTURE    0
ACTUAL_DEP_TIME        0
SCHEDULED_ARRIVAL      0
Temperature            0
Dew Point              0
Humidity               0
Wind                   0
Wind Speed             0
Wind Gust              0
Pressure               0
Condition              0
FLT_SCH_ARRIVAL        0
FLT_SCH_DEPARTURE      0
TAXI_OUT               0
dtype: int64

#### Aggregating using: min, max, means, sum, distinct and count.

In [48]:
pd.set_option('display.max_columns', 120) # configure dataframe to show all columns

dest = taxi.drop(columns=['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']) # create new dataframe from taxi and drop the date columns

dest.groupby(['DEST']).describe() # creates dataframe grouped by destinations, showing details on relevant columns' numerical data.  

DEP_DELAY                                                      SCHEDULED_DURATION                                                          DISTANCE                                                      SCHEDULED_DEPARTURE                                                                  ACTUAL_DEP_TIME                                                                    SCHEDULED_ARRIVAL                                                                  Temperature                                                     Humidity                                                       Wind Speed                                                    Wind Gust                                                Pressure                                                              FLT_SCH_ARRIVAL                                                      FLT_SCH_DEPARTURE                                                     TAXI_OUT                                                   
         count       mean        std   min   25%  50%    75%    max              count        mean       std    min    25%    50%    75%    max    count    mean  std     min     25%     50%     75%     max               count         mean         std     min     25%     50%     75%     max           count         mean         std     min      25%     50%      75%     max             count         mean         std     min     25%     50%     75%     max       count       mean       std   min    25%   50%   75%   max    count       mean        std   min    25%   50%    75%   max      count       mean       std  min   25%   50%    75%   max     count      mean        std  min  25%  50%  75%   max    count       mean       std    min      25%     50%      75%    max           count       mean        std   min   25%   50%    75%   max             count       mean       std   min   25%   50%    75%   max    count       mean       std   min   25%   50%   75%   max
DEST                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
ABQ       58.0  10.327586  39.259098 -13.0 -6.75 -3.0   7.00  215.0               58.0  310.258621  3.522162  307.0  307.0  307.0  314.0  314.0     58.0  1826.0  0.0  1826.0  1826.0  1826.0  1826.0  1826.0                58.0  1195.844828    4.697150  1189.0  1193.0  1200.0  1200.0  1200.0            58.0  1206.172414   40.006200  1178.0  1190.00  1194.0  1203.25  1415.0              58.0  1386.103448    1.682622  1383.0  1387.0  1387.0  1387.0  1387.0        58.0  41.758621  7.472312  26.0  36.00  42.0  48.0  56.0     58.0  62.327586  23.869814  10.0  49.25  61.0  84.25  97.0       58.0  11.948276  5.325946  0.0  7.25  12.0  16.00  25.0      58.0  5.551724  11.050399  0.0  0.0  0.0  0.0  33.0     58.0  30.083966  0.305424  29.38  29.8925  30.095  30.2975  30.73            58.0  37.189655   6.974784  19.0  33.0  40.0  41.75  47.0              58.0  31.465517  5.737582  13.0  28.0  31.0  33.75  43.0     58.0  22.620690  8.243717  11.0  15.0  21.0  29.0  41.0
ATL      795.0   6.725786  26.290094 -13.0 -4.00 -1.0   6.50  341.0              795.0  158.148428  6.547875  136.0  

In [49]:
taxi.groupby(['DEST']).count() # show the counts of each feature for each destination

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
DEST,,,,,,,,,,,,,,,,,,,,,,
ABQ,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58
ATL,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795
AUS,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485
BNA,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366
BOS,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SMF,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82
SRQ,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105
STT,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62


In [50]:
pd.set_option('display.max_columns', 120) # configure dataframe to show all columns

condition = taxi.drop(columns=['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']) # create new dataframe from taxi and drop the date columns

condition.groupby(['Condition']).describe() # creates dataframe grouped by Conditions, showing details on relevant columns' numerical data.  

DEP_DELAY                                                           SCHEDULED_DURATION                                                              DISTANCE                                                                     SCHEDULED_DEPARTURE                                                                  ACTUAL_DEP_TIME                                                                 SCHEDULED_ARRIVAL                                                                   Temperature                                                      Humidity                                                       Wind Speed                                                      Wind Gust                                                     Pressure                                                                 FLT_SCH_ARRIVAL                                                       FLT_SCH_DEPARTURE                                                     TAXI_OUT                                                      
                          count       mean         std   min    25%   50%     75%     max              count        mean         std    min     25%    50%     75%    max    count         mean          std     min      25%     50%      75%     max               count         mean         std    min     25%     50%      75%     max           count        mean         std    min     25%     50%      75%     max             count         mean         std    min      25%     50%      75%     max       count       mean        std   min    25%   50%   75%   max    count       mean        std   min    25%   50%    75%   max      count       mean       std   min    25%   50%    75%   max     count       mean        std   min   25%   50%   75%   max    count       mean           std    min     25%     50%      75%    max           count       mean        std   min    25%   50%    75%   max             count       mean       std   min    25%   50%   75%   max    count       mean        std   min    25%   50%    75%   max
Condition                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
Cloudy                   4992.0   5.316106   38.055821 -22.0  -6.00  -3.0    0.00  1048.0             4992.0  225.112380  117.825273   57.0  125.00  189.5  365.00  697.0   4992.0  1264.958934   875.580354    94.0   483.00  1041.0  2248.00  4983.0              4992.0   825.221554  304.417077  301.0  531.75   857.0  1095.00  1439.0          4992.0  818.999199  311.664436    2.0  523.00   848.0  1092.00  1440.0            4992.0   909.312300  345.897879    1.0   662.00   917.0  1194.00  1439.0      4992.0  43.303486   7.217202  21.0  39.00  43.0  48.0  61.0   4992.0  63.158654  27.643813  10.0  52.00  70.0  85.25  97.0     4992.0   9.875401  4.390900   0.0   7.00   9.0  13.00  20.0    4992.0   0.990184   4.848259   0.0   0.0   0.0   0.0  30.0   4992.0  30.036973  3.143069e-01  29.22  29.810  30.020  30.2825  30.74          4992.0  30.476362   9.926571   0.0  25.00  30.0  37.00  54.0            4992.0  28.195913  8.576998   0.0  20.00

### Ordering the dataset by: date (month, day of month, day of week), carrier_code

In [51]:
# Sorting values by date
taxi.sort_values(by=['MONTH', 'DAY_OF_WEEK', 'DAY_OF_MONTH'])

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEST,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
20511,1,6,1,B6,N952JB,FLL,-8,185,1069,301,293,486,33,29,85,W,10,0,29.93,Cloudy,1,4,31
20512,1,6,1,B6,N973JT,SJU,-7,223,1598,301,294,584,33,29,85,W,10,0,29.93,Cloudy,1,4,29
20513,1,6,1,B6,N982JB,LAX,-3,386,2475,330,327,536,33,29,85,W,10,0,29.93,Cloudy,10,24,25
20514,1,6,1,DL,N110DU,SLC,-1,330,1990,330,329,540,33,29,85,W,10,0,29.93,Cloudy,10,24,34
20515,1,6,1,AA,N176UW,PHX,-3,365,2153,345,342,590,33,29,85,W,10,0,29.93,Cloudy,10,24,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18414,12,29,7,B6,N317JB,BOS,18,70,187,1360,1378,1430,44,44,10,E,12,0,30.00,Light Rain,18,35,24
18415,12,29,7,AA,N117AN,BOS,-5,76,187,1390,1385,26,44,44,10,E,12,0,30.00,Light Rain,22,19,22
18416,12,29,7,B6,N957JB,SJU,0,225,1598,1412,1412,257,44,44,10,E,12,0,30.00,Light Rain,22,19,18
18417,12,29,7,DL,N994AT,TPA,244,190,1005,1169,1413,1359,44,44,10,E,12,0,30.00,Light Rain,22,19,10


In [52]:
# Sorting values by carrier code
taxi.sort_values(by='CARRIER_CODE')

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEST,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
21543,1,9,4,9E,N907XJ,RDU,-5,128,427,535,530,663,26,9,49,NW,12,22,30.73,Fair,48,20,24
18923,12,31,2,9E,N820AY,IAD,37,100,228,1165,1202,1265,43,39,86,SSW,10,0,29.63,Cloudy,23,35,24
24783,1,19,7,9E,N8783E,BWI,-5,86,184,1160,1155,1246,36,18,48,WNW,16,0,29.89,Fair,38,28,14
9459,11,30,6,9E,N600LR,PIT,-8,106,340,454,446,560,32,21,64,N,10,0,30.09,Mostly Cloudy,36,31,28
24791,1,19,7,9E,N8974C,ORF,-3,107,290,1170,1167,1277,34,16,48,WNW,17,28,29.92,Fair,38,28,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26031,1,23,4,YX,N133HQ,CMH,-10,121,483,933,923,1054,42,27,55,SSW,5,0,30.33,Cloudy,30,39,22
26026,1,23,4,YX,N404YX,IND,-11,143,665,930,919,1073,42,27,55,SSW,5,0,30.33,Cloudy,30,39,18
20165,1,4,6,YX,N214JQ,BOS,-2,89,187,1260,1258,1349,47,42,83,NW,18,26,29.53,Light Rain,35,43,39
20166,1,4,6,YX,N875RW,PIT,14,117,340,1250,1264,1367,47,42,83,NW,18,26,29.53,Light Rain,30,27,35


### Dropping columns (dates, categorical data) for k-Fold Cross-Validation

In [53]:
taxiCV = taxi.drop(columns=['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'CARRIER_CODE', 'FLIGHT_NO', 'DEST', 'Wind', 'Condition'])
taxiCV

,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
0,-1,124,636,324,323,448,48,34,58,25,38,29.86,9,17,14
1,-7,371,2475,340,333,531,48,34,58,25,38,29.86,9,17,15
2,40,181,1069,301,341,482,48,34,58,25,38,29.86,9,17,22
3,-2,168,944,345,343,513,48,34,58,25,38,29.86,9,17,12
4,-4,139,760,360,356,499,46,32,58,24,35,29.91,9,17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28815,2,57,150,1370,1372,1427,39,38,96,6,0,30.18,20,32,19
28816,2,75,187,1390,1392,25,39,38,96,6,0,30.18,19,23,22
28817,283,392,2422,1125,1408,1337,39,38,96,6,0,30.18,19,23,21
28818,5,224,1598,1417,1422,261,39,38,96,6,0,30.18,19,23,13


### k-Fold Cross-Validation (CV)

In [54]:
X = taxiCV.iloc[:-1, 0:14] # set predicting variables (initially trying all of them except taxi out time, k = n) from taxiCV
y = taxiCV.iloc[:-1, 14] # set variable of what we want to predict (taxi out time)

# Define model evaluation method with 15 fold cross-validation
cv = RepeatedKFold(n_splits=15, n_repeats=3, random_state=1) 

# Defining the model with 3 alphas of 0, 1, and 0.01 (λ), one for each n_repeat
taxi_model = RidgeCV(alphas=arange(0, 1, 0.01), cv=cv, scoring='neg_mean_absolute_error') 

# Fitting the model
taxi_model.fit(X, y) 

# Summarise chosen configuration
print('alpha: %f' % taxi_model.alpha_) # λ = 0.99

alpha: 0.990000


### Splitting the dataset (90:10)
> See Real Python's [guide](https://realpython.com/train-test-split-python-data/) for splitting data with scikit-learn and python.
 
> See [this](https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas) stack overflow thread for examples using pandas data frames. 

In [55]:
# Using X and y from k-Fold CV above, split dataset with ratio of train to test of 90:10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1) # test_size set to 0.1 because we want 0.9 train_size, random_state can be any integer

In [56]:
X_train # Show X_train

,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE
12992,-1,105,427,1030,1029,1135,31,17,57,3,0,30.71,32,32
27682,-3,187,740,970,967,1097,41,25,53,18,0,29.83,28,29
16171,-3,365,2153,345,342,590,35,31,85,8,0,30.14,11,21
15508,30,263,1521,1289,1319,52,27,15,61,10,0,30.63,32,35
4622,-14,192,1089,485,471,677,40,27,60,8,0,30.26,36,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10955,-10,100,266,1320,1310,1420,39,25,57,15,30,30.02,29,32
17291,-2,292,1626,1015,1013,1187,45,40,82,14,0,30.36,28,32
5192,-5,412,2586,1199,1194,1431,34,5,30,10,0,30.46,40,26
12172,-2,384,2475,1320,1318,84,50,50,10,14,0,29.77,26,28


In [57]:
y_train # Show y_train

12992    26
27682    18
16171    21
15508    12
4622     26
         ..
10955    35
17291     9
5192     25
12172    14
235      23
Name: TAXI_OUT, Length: 25935, dtype: int64

In [58]:
X_test # Show X_test

,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE
1637,-6,87,213,515,509,602,50,33,52,12,0,30.39,52,18
22484,-3,265,1391,539,536,744,62,49,62,25,0,29.98,49,18
14604,103,85,187,1034,1137,1119,34,34,10,12,0,29.71,38,36
5945,-11,379,2475,425,414,624,43,43,10,10,0,29.78,16,14
26858,0,125,427,516,516,641,40,35,83,13,0,29.83,47,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184,8,128,483,1220,1228,1348,30,7,38,9,0,30.43,29,44
938,-11,180,1005,447,436,627,44,36,73,3,0,30.37,35,28
25913,-7,395,2422,546,539,761,38,28,68,9,0,30.43,46,18
5549,-8,83,266,1320,1312,1403,39,31,73,17,0,30.07,32,34


In [59]:
y_test # Show y_test

1637     20
22484    33
14604    31
5945     19
26858    16
         ..
4184     14
938      10
25913    15
5549     21
8351     27
Name: TAXI_OUT, Length: 2882, dtype: int64

In [60]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape) # Show sizes of train and test data

(25935, 14) (2882, 14) (25935,) (2882,)


### Creating the Linear Regression 
> **Source:** See DHWANIT BALWANI'S [work](https://realpython.com/train-test-split-python-data/) on Kaggle. 

In [74]:

# Create a linear regression object called lreg
lreg = LinearRegression()  

# Perform the linear regression using X_train and y_train
lreg.fit(X_train, y_train)  

# Make predictions based on the model
y_pred = lreg.predict(X_test)  


# Calculate the mean squared error, **0.5 will square root the mean squared error
rmse = mean_squared_error(y_test,y_pred)**0.5
print(rmse) # rmse = 6.575824000116706

6.575824000116706


### Ridge Regression

In [77]:
# define model
ridge_mod = Ridge(alpha=0.05)

# fit model
ridge_mod.fit(X_train, y_train)

# make prediction
ridge_pred = ridge_mod.predict(X_test)

# Calculate the mean squared error
rmser = mean_squared_error(y_test,y_pred)**0.5
print(rmser) #rmser = 6.575824000116706 when alpha is 0.05)



6.575824000116706


### Lasso Regression

In [78]:
# define model
lasso_mod = Lasso(alpha=0.05)

# fit model
lasso_mod.fit(X_train, y_train)

# make prediction
lasso_pred = lasso_mod.predict(X_test)

# Calculate the mean squared error
rmsel = mean_squared_error(y_test,y_pred)**0.5
print(rmsel) #rmsel = 6.575824000116706 when alpha is 0.05)


6.575824000116706
